### Se importan las Librerias

In [1]:
import pandas as pd

Se extraen e importan 2 dataframes y se almacenan en variables diferentes, el cual se lee desde la wiki fandom del honkai

In [2]:
characterUrl = pd.read_html('https://honkai-star-rail.fandom.com/wiki/Character')
playableCharacters = characterUrl[1]
upcomingCharacters = characterUrl[2]

playableCharacters.head()

,Icon,Name,Rarity,Path,Combat Type
0,NaN,Arlan,NaN,Destruction,Lightning
1,NaN,Asta,NaN,Harmony,Fire
2,NaN,Bailu,NaN,Abundance,Lightning
3,NaN,Blade,NaN,Destruction,Wind
4,NaN,Bronya,NaN,Harmony,Wind


Se dropean columnas no relevantes e innecesarias

In [3]:
playableCharacters.drop(columns=['Icon', 'Rarity'], inplace=True)
upcomingCharacters.drop(columns=['Icon', 'Rarity'], inplace=True)
playableCharacters.head()

,Name,Path,Combat Type
0,Arlan,Destruction,Lightning
1,Asta,Harmony,Fire
2,Bailu,Abundance,Lightning
3,Blade,Destruction,Wind
4,Bronya,Harmony,Wind


In [4]:
upcomingCharacters.head()

,Name,Path,Combat Type
0,Argenti,Erudition,Physical
1,Dr. Ratio,The Hunt,Imaginary
2,Hanya,Harmony,Physical
3,Ruan Mei,Harmony,Ice
4,Xueyi,Destruction,Quantum


Se modifica el codigo aca para agregar 1 columna a cada df, para poder diferenciar si son jugables o no actualmente para un posterior analisis

In [5]:
playableCharacters['Status'] = 'Playable'
upcomingCharacters['Status'] = 'Upcoming'

In [6]:
upcomingCharacters.head()


,Name,Path,Combat Type,Status
0,Argenti,Erudition,Physical,Upcoming
1,Dr. Ratio,The Hunt,Imaginary,Upcoming
2,Hanya,Harmony,Physical,Upcoming
3,Ruan Mei,Harmony,Ice,Upcoming
4,Xueyi,Destruction,Quantum,Upcoming


In [7]:
playableCharacters.head()


,Name,Path,Combat Type,Status
0,Arlan,Destruction,Lightning,Playable
1,Asta,Harmony,Fire,Playable
2,Bailu,Abundance,Lightning,Playable
3,Blade,Destruction,Wind,Playable
4,Bronya,Harmony,Wind,Playable


Se unen los df para tener un consolidado con los personajes jugables y los proximos

In [8]:
characters_df = pd.concat([playableCharacters, upcomingCharacters], ignore_index=False)
characters_df.sample(10)

,Name,Path,Combat Type,Status
6,Dan Heng,The Hunt,Wind,Playable
9,Gepard,Preservation,Ice,Playable
1,Asta,Harmony,Fire,Playable
0,Arlan,Destruction,Lightning,Playable
35,Yukong,Harmony,Imaginary,Playable
11,Herta,Erudition,Ice,Playable
13,Hook,Destruction,Fire,Playable
30,Tingyun,Harmony,Lightning,Playable
16,Jingliu,Destruction,Ice,Playable
0,Argenti,Erudition,Physical,Upcoming


In [9]:
characters_df2 = characters_df.reset_index(drop=True, inplace=False)
characters_df2.sample(10)

,Name,Path,Combat Type,Status
6,Dan Heng,The Hunt,Wind,Playable
40,Xueyi,Destruction,Quantum,Upcoming
18,Luka,Nihility,Physical,Playable
0,Arlan,Destruction,Lightning,Playable
24,Qingque,Erudition,Quantum,Playable
29,Sushang,The Hunt,Physical,Playable
34,Yanqing,The Hunt,Ice,Playable
21,March 7th,Preservation,Ice,Playable
22,Natasha,Abundance,Physical,Playable
23,Pela,Nihility,Ice,Playable


Aca pasan muchas cosas. Primero se crea un array vacio, despues se buclean todos los nombres que existan en Name dentro del df characters_df2. Se genera una url dinamica en donde se intenta extraer tabla con informacion y stats de los personajes. Se generan transformaciones dentro y fuera del bucle para que no genere errores. Las tablas no encontradas se printean junto a la url a la que se intento acceder.

In [41]:
dfs = []

for name in characters_df2['Name']:

    url_name = name.replace(' ', '_')
    url = 'https://honkai-star-rail.fandom.com/wiki/' + url_name + '/Combat'

    try:
        characterStatsUrl = pd.read_html(url)
        characterStats = characterStatsUrl[0]
        characterStats['Name'] = name
        characterStats.drop(columns=['AscensionPhase'], inplace=True)
        dfs.append(characterStats)

    except:
        print('No se encuentran tablas en la página de ' + name + ' '+ url)


charactersStats_df = pd.concat(dfs, ignore_index=False)
charactersStats_df.dropna(inplace=True)
charactersStats_df.drop(columns=['Ascension Costs'], inplace=True)
charactersStats_df = charactersStats_df[charactersStats_df['Level'] == '80/80']
charactersStats_df.replace(to_replace='80/80', value='80', inplace=True)
charactersStats_df.reset_index(drop=True, inplace=True)
charactersStats_df.sample(10)



No se encuentran tablas en la página de Dan Heng • Imbibitor Lunae https://honkai-star-rail.fandom.com/wiki/Dan_Heng_•_Imbibitor_Lunae/Combat
No se encuentran tablas en la página de Argenti https://honkai-star-rail.fandom.com/wiki/Argenti/Combat
No se encuentran tablas en la página de Dr. Ratio https://honkai-star-rail.fandom.com/wiki/Dr._Ratio/Combat
No se encuentran tablas en la página de Hanya https://honkai-star-rail.fandom.com/wiki/Hanya/Combat
No se encuentran tablas en la página de Ruan Mei https://honkai-star-rail.fandom.com/wiki/Ruan_Mei/Combat
No se encuentran tablas en la página de Xueyi https://honkai-star-rail.fandom.com/wiki/Xueyi/Combat


,Level,BaseHP,BaseATK,BaseDEF,BaseSPD,Name
19,80,1058,493,551,100,Lynx
12,80,1340,617,352,94,Hook
10,80,952,582,396,100,Herta
22,80,987,546,463,105,Pela
31,80,1203,620,460,100,Trailblazer
20,80,1058,511,573,101,March 7th
33,80,892,679,412,109,Yanqing
1,80,1023,511,463,106,Asta
21,80,1164,476,507,98,Natasha
2,80,1319,562,485,98,Bailu


Se mergea la tabla nueva con stats de los personajes junto  a la tabla de PLAYABLE CHARACTERS, ya que los personajes no encontrados en el paso anterior coinciden a los upcoming. (se podria usar el df llamado characters_df2 igualmente)

In [42]:
charactersPlayables_df = pd.merge(playableCharacters, charactersStats_df, on='Name')
charactersPlayables_df.sample(10)

,Name,Path,Combat Type,Status,Level,BaseHP,BaseATK,BaseDEF,BaseSPD
8,Gepard,Preservation,Ice,Playable,80,1397,543,654,92
30,Topaz and Numby,The Hunt,Fire,Playable,80,931,620,412,110
31,Trailblazer,Adaptive,Adaptive,Playable,80,1203,620,460,100
32,Welt,Nihility,Imaginary,Playable,80,1125,620,509,102
6,Dan Heng,The Hunt,Wind,Playable,80,882,546,396,110
2,Bailu,Abundance,Lightning,Playable,80,1319,562,485,98
17,Luka,Nihility,Physical,Playable,80,917,582,485,103
3,Blade,Destruction,Wind,Playable,80,1358,543,485,97
26,Serval,Erudition,Lightning,Playable,80,917,652,374,104
18,Luocha,Abundance,Imaginary,Playable,80,1280,756,363,101


Se genera un csv para poder importar directamente estos datos y analizarlos en Looker

In [43]:
charactersPlayables_df.to_csv('Honkai-Star-Rail-EDA-df.csv', index=False)